In [9]:
#Library
import pandas as pd 
from pmaw import PushshiftAPI
import datetime as dt
import requests
import json

In [10]:
api = PushshiftAPI()

In [11]:
#convert date to epoch time 

#before = int(dt.datetime(2021,3,2,0,0).timestamp()) 
start_time =  int(dt.datetime(2021,3,3,0,0).timestamp())


In [14]:
#convert date to epoch time 

#before = int(dt.datetime(2021,3,2,0,0).timestamp()) 
start_time =  int(dt.datetime(2021,3,2,0,0).timestamp())
wsb = "wallstreetbets"
limit = 10

submissions = api.search_submissions(subreddit=wsb, 
                                     limit=limit,
                                     after=start_time,
                                     filter=['title', 'selftext', 'id' , 'subreddit', 'url', 'removed_by_category']
                                     ) #'author'

#print(f'Retrieved {len(submissions)} comments from Pushshift')
submissions_df = pd.DataFrame(submissions)

Total:: Success Rate: 100.00% - Requests: 2 - Batches: 1 - Items Remaining: 0


In [16]:
#submissions_df.head()

In [17]:
post_id = list(submissions_df.id)
#post_id

In [19]:
#get all the comments ids for our submissions
comment_ids = api.search_submission_comment_ids(ids=post_id)

comment_id_list = [c_id for c_id in comment_ids]
len(comment_id_list)

Total:: Success Rate: 100.00% - Requests: 8 - Batches: 1 - Items Remaining: 0


10

In [20]:
#Take the comments ids and find the body. 
comments = api.search_comments(ids=comment_id_list,
                               limit=1000,
                               filter=['author', 'body', 'subreddit', 'id', 'link_id'])
#comment_list = [comment for comment in comments]
df_comments = pd.DataFrame(comments)
df_comments

Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1 - Items Remaining: 0


,author,body,id,link_id,subreddit
0,AutoModerator,Your post has been removed because a single li...,gpgqoji,t3_lwdn0t,wallstreetbets
1,Traditional_Base_584,"Imma let my phone die, so that way i do not lo...",gpgqr4v,t3_lwdmwz,wallstreetbets
2,toxicsleft,Drawing attention to that Edit for those of yo...,gpgqrmt,t3_lwdn0t,wallstreetbets
3,kamajie,33% it dips\n33% it goes up\n33% it goes sidew...,gpgqvdh,t3_lwdn75,wallstreetbets
4,Motorbreath91,ARISE CHICKEN!,gpgr022,t3_lwdn1f,wallstreetbets
5,Neat-Answer-3630,What show is that again? Robot chicken?,gpgt7le,t3_lwdn1f,wallstreetbets
6,ijustwanttowant,Aqua teen hunger force,gpgup9l,t3_lwdn1f,wallstreetbets
7,ChoboJawz,Bad reception. We go over here.... Arise Chick...,gpgxg75,t3_lwdn1f,wallstreetbets
8,Jeffery_G,My name is Shake Zula!,gpgyxpe,t3_lwdn1f,wallstreetbets
9,QuirkyResponsibility,"Chicken, arise!",gpgzgbv,t3_lwdn1f,wallstreetbets


In [21]:
#grouping comments for each posts and concatenating them together
#One row per post, with that row con taining all the comments on that post. 

grouped_comments = df_comments.groupby('link_id').agg({'body': '\n'.join})
grouped_comments.head()

,body
link_id,
t3_lwdmwz,"Imma let my phone die, so that way i do not lo..."
t3_lwdn0t,Your post has been removed because a single li...
t3_lwdn1f,ARISE CHICKEN!\nWhat show is that again? Robot...
t3_lwdn75,33% it dips\n33% it goes up\n33% it goes sidew...


In [22]:
# join the comments and the posts 
# set_index to join the id of the post with the link_id of the comment 

df_test = submissions_df.set_index('id').join(grouped_comments,)

df_test

,created_utc,selftext,subreddit,title,url,removed_by_category,body
id,,,,,,,
lwdn75,1614722303,Is there a high probability that $RKT will dip...,wallstreetbets,QUESTION: What are the chances that $RKT will ...,https://www.reddit.com/r/wallstreetbets/commen...,NaN,NaN
lwdn6i,1614722302,,wallstreetbets,"Okay, which one of you lads did I have the ple...",https://i.redd.it/rnwn7apwsok61.jpg,moderator,NaN
lwdn1k,1614722290,[removed],wallstreetbets,$rkt gamma squeeze on Friday March 5 if price ...,https://www.reddit.com/r/wallstreetbets/commen...,moderator,NaN
lwdn1f,1614722290,,wallstreetbets,I love you all &lt;3,https://v.redd.it/gcdxkzo1sok61,NaN,NaN
lwdn0t,1614722289,,wallstreetbets,"Dear RKT, reposted important clip from DD here.",https://i.redd.it/yu5dhsuusok61.jpg,moderator,NaN
lwdmzx,1614722286,[removed],wallstreetbets,RKT earning move to GME,https://www.reddit.com/r/wallstreetbets/commen...,moderator,NaN
lwdmzu,1614722286,[removed],wallstreetbets,Tomorrow RKT 🚀🚀,https://www.reddit.com/r/wallstreetbets/commen...,moderator,NaN
lwdmwz,1614722280,,wallstreetbets,All in and fuck it,https://i.redd.it/bncq6uiusok61.jpg,NaN,NaN
lwdmo5,1614722262,[removed],wallstreetbets,Dear strangers,https://www.reddit.com/r/wallstreetbets/commen...,moderator,NaN
